# Installs

In [1]:
from google.colab import drive
import sys
drive.mount('/mnt/drive')

storage_path = "/mnt/drive/MyDrive/cs224u_final_project"
sys.path.append(storage_path)

Mounted at /mnt/drive


In [2]:
%%capture
!pip install wget openai datasets pandas pyarrow

# Imports

In [3]:
import pandas as pd
import pyarrow as pa
from datasets import Dataset

import os
import wget
import re
from openai import OpenAI
client = OpenAI(api_key="")

# Load Recogs data

In [4]:
SRC_DIRNAME = os.path.join("data", "recogs")

if not os.path.exists(SRC_DIRNAME):
    os.makedirs('data', exist_ok=True)
    wget.download('https://web.stanford.edu/class/cs224u/data/recogs.tgz', out='data/')
    !tar xvf data/recogs.tgz -C data/

recogs/
recogs/._train.tsv
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
recogs/train.tsv
recogs/._tgt_vocab.txt
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
recogs/tgt_vocab.txt
recogs/._decoder_config.json
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
recogs/decoder_config.json
recogs/._dev.tsv
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
recogs/dev.tsv
recogs/._encoder_config.json
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
recogs/encoder_config.json
recogs/._src_vocab.txt
tar: Ignoring unknown extended header keyw

In [5]:
def load_split(filename):
    return pd.read_csv(
        filename,
        delimiter="\t",
        names=['input', 'output', 'category'])

dataset = {}

for splitname in ("train", "dev", "gen"):
    dataset[splitname] = load_split(f"{SRC_DIRNAME}/{splitname}.tsv")

In [6]:
def standardize_logical_form_numbering(sentence: str) -> str:
  pattern = re.compile("\(\s(\d+)\s\)")
  new_sentence = sentence
  string_numbers = pattern.findall(sentence)
  integer_numbers = [int(item) for item in string_numbers]

  replace_dict = dict(zip(
    [str(item) for item in integer_numbers],
    [str(item) for item in range(1,len(integer_numbers)+1)],
  ))
  for old, new in replace_dict.items():
    new_sentence = new_sentence.replace(f" {old} ", f" {new} ")
  return new_sentence

In [7]:
for stage in ['train', 'dev', 'gen']:
  dataset[stage]['output_renum'] = dataset[stage]['output'].apply(standardize_logical_form_numbering)
dataset['train'].head()

,input,output,category,output_renum
0,A rose was helped by a dog .,rose ( 53 ) ; dog ( 38 ) ; help ( 7 ) AND them...,in_distribution,rose ( 1 ) ; dog ( 2 ) ; help ( 3 ) AND theme ...
1,The sailor dusted a boy .,* sailor ( 48 ) ; boy ( 53 ) ; dust ( 10 ) AND...,in_distribution,* sailor ( 1 ) ; boy ( 2 ) ; dust ( 3 ) AND ag...
2,Emma rolled a teacher .,Emma ( 17 ) ; teacher ( 50 ) ; roll ( 39 ) AND...,in_distribution,Emma ( 1 ) ; teacher ( 2 ) ; roll ( 3 ) AND ag...
3,Evelyn rolled the girl .,Evelyn ( 46 ) ; * girl ( 16 ) ; roll ( 21 ) AN...,in_distribution,Evelyn ( 1 ) ; * girl ( 2 ) ; roll ( 3 ) AND a...
4,A cake was forwarded to Levi by Charlotte .,cake ( 5 ) ; Levi ( 34 ) ; Charlotte ( 33 ) ; ...,in_distribution,cake ( 1 ) ; Levi ( 2 ) ; Charlotte ( 3 ) ; fo...


# Define metric

In [4]:
from compgen import recogs_exact_match

# Get step by step examples and instruction from GPT-4

In [1]:
# prompt = """You must come up with a general step by step guide, how to create the logical form from a given sentence.
# The guide should also explain the syntax and symbols that the specific logical form requires.
# The guide must be compact and use only as many words as are needed to cover all necessary rules. It must not contain any markdown formatting.
# The last step of the guide should only return the full correct logical form.

# Using one of the example pairs, give a demonstration how you use the guide to convert a sentence into its correct logical form.

# ---
# Examples:

# """

# sample_df = dataset["train"].sample(n=20)

# example_strings = sample_df.apply(lambda row: f"Sentence: {row['input']}\nLogical Form: {row['output_renum']}\n", axis=1).to_list()

# prompt = prompt + "\n".join(example_strings)
# prompt = prompt + """
# ---
# Step by step guide:"""

# response = client.chat.completions.create(
#   model="gpt-4-0125-preview",
#   messages=[
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": prompt},
#   ]
# )

# print(response.choices[0].message.content)

In [4]:
instructions = """1. Identify all entities mentioned in the sentence. Assign a unique identifier to each, starting with 1. Mark entities with asterisks if they are introduced by "The" without specific names.
2. Locate the main verb or verbs that describe actions, states, or occurrences. Assign identifiers continuing from the last entity.
3. Use syntax to create relations:
   - `agent(verb_id, entity_id)` indicates the doer of the action.
   - `theme(verb_id, entity_id)` indicates what or whom the action is directed towards.
   - `recipient(verb_id, entity_id)` for indirect objects receiving the direct object.
   - `nmod.prep(related_entity_id, entity_id)` for prepositional phrases, where `prep` is the preposition.
   - `ccomp(verb_id, secondary_action_id)` for clauses that the verb comprehends.
   - `xcomp(verb_id, secondary_action_id)` for clauses that are an extension of the action.
4. Combine relations using `AND` to form the complete logical form.
5. Return the logical form.

Symbols and syntax rules:
   - Entities and actions are separated by semicolons (`;`).
   - Relations are listed after actions, separated by `AND`.
   - Parentheses group the verb or modifier with the involved entities (`relation(id, id)`).
   - Asterisks (`*`) mark definite entities without specific identification.

---
Demonstration with example:

Sentence: The boy returned the girl the raisin in the house.

1. Identify entities: * boy ( 1 ) ; * girl ( 2 ) ; * raisin ( 3 ) ; * house ( 4 )
2. Locate main verb/action: returned ( 5 )
3. Create relations:
   - `agent ( 5 , 1 )` for "The boy" who does the returning.
   - `recipient ( 5 , 2 )` for "the girl" who receives.
   - `theme ( 5 , 3 )` for "the raisin" being returned.
   - `nmod.in ( 3 , 4 )` for "in the house" modifying where "the raisin" is.
4. Combine relations: `* boy ( 1 ) ; * girl ( 2 ) ; * raisin ( 3 ) ; * house ( 4 ) ; nmod.in ( 3 , 4 ) AND return ( 5 ) AND agent ( 5 , 1 ) AND recipient ( 5 , 2 ) AND theme ( 5 , 3 )`
5. Return the logical form:
* boy ( 1 ) ; * girl ( 2 ) ; * raisin ( 3 ) ; * house ( 4 ) ; nmod . in ( 3 , 4 ) AND return ( 5 ) AND agent ( 5 , 1 ) AND recipient ( 5 , 2 ) AND theme ( 5 , 3 )"""

In [39]:
# system_prompt = f"""You are given a sentence and a specific logical form that represents the semantics of the sentence.
# Your task is to follow a step by step guide and generate answers for each of the steps, while using as few words as possible.
# The answer to each step should lead to the given logical form, which you return fully in the last step.
# For support, you are also given a demonstration of how to use the step by step guide.

# ---
# Step by step guide and demonstration:
# {instructions}"""

# training_examples_df = dataset["train"].sample(500)
# generated_examples = []

# count = 0
# for id, row in training_examples_df.iterrows():
#   if count % 25 == 0:
#     print(count)
#   response = client.chat.completions.create(
#     model="gpt-3.5-turbo-0125", #"gpt-4-0125-preview",
#     messages=[
#       {"role": "system", "content": system_prompt},
#       {"role": "user", "content": f"Sentence: {row['input']}\nLogical Form: {row['output_renum']}"},
#     ]
#   )
#   generated_examples.append(
#       {'sentence': row['input'],
#       'logical_form': row['output_renum'],
#       'guide': response.choices[0].message.content}
#   )
#   training_examples_df.loc[id, "gpt35_guide"] = response.choices[0].message.content
#   count += 1

0
25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475


In [40]:
# training_examples_df.to_parquet(f"{storage_path}/gpt35_examples_500.parquet")

In [41]:
# import pickle
# with open(f"{storage_path}/gpt35_examples_500.pickle", "wb") as file:
#   pickle.dump(generated_examples, file)

# with open(f"{storage_path}/gpt35_examples_500.pickle", "rb") as file:
#   generated_examples = pickle.load(file)

# Load data on rerun and mark faulty

In [6]:
training_examples_df = pd.read_parquet(f"{storage_path}/gpt35_examples_500.parquet")
training_examples_df.head()

,input,output,category,output_renum,gpt35_guide
125472,Jayden gave Emma the balloon the cookie was lo...,Jayden ( 38 ) ; Emma ( 17 ) ; * balloon ( 41 )...,length_ood,Jayden ( 1 ) ; Emma ( 2 ) ; * balloon ( 3 ) ; ...,1. Identify entities: Jayden (1); Emma (2); *b...
124055,Liam hoped that the dog preferred to run Sophi...,Liam ( 0 ) ; * dog ( 23 ) ; Sophia ( 50 ) ; Ja...,length_ood,Liam ( 1 ) ; * dog ( 2 ) ; Sophia ( 3 ) ; Jack...,1. Identify entities: Liam ( 1 ) ; * dog ( 2 )...
78031,Liam ran .,"Liam ( 52 ) ; run ( 26 ) AND agent ( 26 , 52 )",in_distribution,"Liam ( 1 ) ; run ( 2 ) AND agent ( 2 , 1 )",1. Identify entities: Liam ( 1 )\n2. Locate ma...
114594,The girl forwarded the melon in a house to Emma .,* girl ( 53 ) ; * melon ( 18 ) ; house ( 41 ) ...,in_distribution,* girl ( 1 ) ; * melon ( 2 ) ; house ( 3 ) ; E...,1. Identify entities: * girl ( 1 ) ; * melon (...
85560,A scientist gave the radio to Emma .,scientist ( 48 ) ; * radio ( 59 ) ; Emma ( 21 ...,in_distribution,scientist ( 1 ) ; * radio ( 2 ) ; Emma ( 3 ) ;...,1. Identify entities: scientist ( 1 ) ; * radi...


In [7]:
training_examples_df["result_guide"] = training_examples_df.gpt35_guide.str.split("6. Return the logical form:").str[-1].str.strip()
training_examples_df["guide_correct"] = training_examples_df.apply(lambda row: recogs_exact_match(row["output_renum"], row["result_guide"]), axis=1)
training_examples_df["guide_correct"].mean()

0.322

In [8]:
training_examples_df["guide_correct"].sum()

161

# Generate reduced test datasets

In [5]:
# dataset["gen"].sample(n=200).to_parquet(f"{storage_path}/test_data_200.parquet")
# dataset["gen"].sample(n=1000).to_parquet(f"{storage_path}/test_data_1000.parquet")

testing_examples_df = pd.read_parquet(f"{storage_path}/test_data_200.parquet")

# Prepare and train model

Adapted from this [notebook](https://colab.research.google.com/drive/1Dyauq4kTZoLewQ1cApceUQVNcnnNTzg_?usp=sharing#scrollTo=2eSvM9zX_2d3)

In [6]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-th23jk_l/unsloth_f5b7da4d36644b26a7ea440183db90d4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-th23jk_l/unsloth_f5b7da4d36644b26a7ea440183db90d4
  Resolved https://github.com/unslothai/unsloth.git to commit a68aebc1fa17755ffbcdafc9239e7ca37ab21657
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 2.5 MB/s eta 0:00:00
  Created wheel for unsloth: filename=unsloth-2024.3-py3-none-any.whl size=93934 sha256=c3bfbeace4bfba7a397333dd8c6d8a91c8d6c8da7436cdd25e9fc259a69631bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-9j1kcsw9/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222

In [7]:
import gc
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
#     "unsloth/llama-2-7b-bnb-4bit",
#     "unsloth/llama-2-13b-bnb-4bit",
#     "unsloth/codellama-34b-bnb-4bit",
#     "unsloth/tinyllama-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
#     "unsloth/gemma-2b-bnb-4bit",
# ] # More models at https://huggingface.co/unsloth

# trainer.accelerator.free_memory()
# del model, tokenizer, trainer
# gc.collect()
# torch.cuda.empty_cache()

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [8]:
system_prompt = f"""You are given a sentence and must convert it to a specific logical form that represents the semantics of the sentence.
To do this you must follow a step by step guide that you are provided with below.
You must follow the steps and generate answers for each of the steps, while using as few words as possible.
The answer to each step should lead to the logical form, which you return in the last step.
For support, you are also given an example of how to use the step by step guide to convert a given sentence to its logical form.


---
Step by step guide and demonstration:
{instructions}


---
Sentence to convert:
"""

##############
# Test set
##############

testing_examples_df["prompt_test_base"] = testing_examples_df.apply(lambda row: tokenizer.apply_chat_template([{"role": "user", "content": row["input"]}
                                                                                                            ], return_tensors="pt", tokenize=False), axis=1)
testing_examples_df["prompt_test_guide"] = testing_examples_df.apply(lambda row: tokenizer.apply_chat_template([{"role": "user", "content": system_prompt + row["input"]}
                                                                                                            ], return_tensors="pt", tokenize=False), axis=1)

testing_examples_df.to_parquet(f"{storage_path}/test_data_200_v2.parquet")
testing_examples_df.head()

##############
# Train set
##############

training_examples_df["prompt_training_base"] = training_examples_df.apply(lambda row: tokenizer.apply_chat_template([{"role": "user", "content": row["input"]},
                                                                                                            {"role": "assistant", "content": " " + row["output_renum"]}
                                                                                                            ], return_tensors="pt", tokenize=False), axis=1)
training_examples_df["prompt_training_guide"] = training_examples_df.apply(lambda row: tokenizer.apply_chat_template([{"role": "user", "content": system_prompt + row["input"]},
                                                                                                            {"role": "assistant", "content": " " + row["gpt35_guide"]}
                                                                                                            ], return_tensors="pt", tokenize=False), axis=1)
training_examples_df["prompt_training_guide_num_token"] = training_examples_df.apply(
    lambda row: tokenizer(row["prompt_training_guide"], padding=False, return_tensors='pt')['input_ids'].shape[1],
    axis=1)
training_examples_df["prompt_training_base_num_token"] = training_examples_df.apply(
    lambda row: tokenizer(row["prompt_training_base"], padding=False, return_tensors='pt')['input_ids'].shape[1],
    axis=1)
training_examples_df["gpt35_guide_num_token"] = training_examples_df.apply(
    lambda row: tokenizer(row["gpt35_guide"], padding=False, return_tensors='pt')['input_ids'].shape[1],
    axis=1)
training_examples_df.to_parquet(f"{storage_path}/gpt35_examples_500_num_tokens.parquet")
training_examples_df.head()

,input,output,category,output_renum,prompt_test_base,prompt_test_guide
9202,A lion squeezed a balloon on the stage .,lion ( 46 ) ; balloon ( 44 ) ; * stage ( 49 ) ...,passive_to_active,lion ( 1 ) ; balloon ( 2 ) ; * stage ( 3 ) ; n...,<s>[INST] A lion squeezed a balloon on the sta...,<s>[INST] You are given a sentence and must co...
20450,Emma squeezed the professor on the hanger .,Emma ( 48 ) ; * professor ( 59 ) ; * hanger ( ...,passive_to_active,Emma ( 1 ) ; * professor ( 2 ) ; * hanger ( 3 ...,<s>[INST] Emma squeezed the professor on the h...,<s>[INST] You are given a sentence and must co...
14358,The girl squeezed Liam .,* girl ( 43 ) ; Liam ( 56 ) ; squeeze ( 20 ) A...,passive_to_active,* girl ( 1 ) ; Liam ( 2 ) ; squeeze ( 3 ) AND ...,<s>[INST] The girl squeezed Liam . [/INST],<s>[INST] You are given a sentence and must co...
12832,The teacher in the cart cooked Emma .,* teacher ( 31 ) ; * cart ( 6 ) ; Emma ( 47 ) ...,obj_pp_to_subj_pp,* teacher ( 1 ) ; * cart ( 2 ) ; Emma ( 3 ) ; ...,<s>[INST] The teacher in the cart cooked Emma ...,<s>[INST] You are given a sentence and must co...
2052,Michael believed that Mason hoped that the con...,Michael ( 15 ) ; Mason ( 37 ) ; * consumer ( 4...,cp_recursion,Michael ( 6 ) ; Mason ( 2 ) ; * consumer ( 3 )...,<s>[INST] Michael believed that Mason hoped th...,<s>[INST] You are given a sentence and must co...


In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
FastLanguageModel.for_training(model)

Unsloth 2024.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [13]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments

num_training = 161
base_or_guide = "guide"
model_name = f"model_{base_or_guide}_{num_training}"

dataset_df = training_examples_df.loc[training_examples_df.guide_correct, :].iloc[:num_training, :]
if base_or_guide == "base":
  # dataset_df = dataset_df[["input", "output_renum"]]
  # train_dataset = Dataset(pa.Table.from_pandas(dataset_df))

  # def formatting_prompts_func(example):
  #   text = f"<s>[INST]{example['input']}[/INST]{example['output_renum']}</s>"
  #   return text
  train_dataset = Dataset(pa.Table.from_pandas(pd.DataFrame({"text": dataset_df.prompt_training_base})))

elif base_or_guide == "guide":
  # dataset_df = dataset_df["input", "gpt4_guide"]
  # train_dataset = Dataset(pa.Table.from_pandas(dataset_df))

  # def formatting_prompts_func(example):
  #   text = f"<s>[INST]{system_prompt + example['input']}[/INST]{example['gpt4_guide']}</s>"
  #   return text
  train_dataset = Dataset(pa.Table.from_pandas(pd.DataFrame({"text": dataset_df.prompt_training_guide})))

# instruction_template = "<s>[INST]"
# response_template = "[/INST]"
collator = DataCollatorForCompletionOnlyLM(tokenizer.encode("\n[/INST]", add_special_tokens = False)[3:],
                                          #  instruction_template=instruction_template,
                                           tokenizer=tokenizer)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    # formatting_func=formatting_prompts_func,
    data_collator=collator,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs=10,
        # max_steps = 200,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        # save_steps=21,
        save_strategy="epoch",
        overwrite_output_dir=True,
        output_dir = f"{storage_path}/{model_name}",
    ),
)

Map (num_proc=2):   0%|          | 0/161 [00:00<?, ? examples/s]

In [14]:
import json

results = trainer.train()

# model.save_pretrained(f"{storage_path}/{model_name}_final")

with open(f"{storage_path}/{model_name}_logs.json", "w") as file:
  json.dump(trainer.state.log_history, file)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 161 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 200
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.245200
2,0.219200
3,0.195800
4,0.152500
5,0.122800
6,0.120300
7,0.129800
8,0.113200
9,0.137600
10,0.082100
